In [24]:
#import dataset class
from dataset import energyProject_dataset
from torch.utils.data import DataLoader
import pickle
import torch
import torch.nn as nn
import numpy as np
import tqdm
import torch
import torch.nn as nn
import pandas as pd
import yaml
from tqdm import tqdm
from icecream import ic

In [3]:
#load dataset object file
with (open('data/dataset_class.pkl', "rb")) as openfile:
    dataset = pickle.load(openfile)

In [4]:
#now you can create a dataloader and use it!
batch_size = 10
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
dataset.activitivity_encoding_mode = 4 #or any value

In [5]:
BERT_DIM_EMB = 768
TIME_CLIMATE_DIM = 21
import torch.nn.init as init

In [6]:
device = "mps"

In [7]:
import torch.optim as optim

In [8]:
from transformers import AutoTokenizer,  DistilBertModel
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
bert.to(device)

/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mod

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [9]:
start_token = "[CLS] "
end_token = " [SEP]"

In [10]:
def txt2tokens(txt):    
    test_txt = start_token + txt + end_token
    tokenized_text = tokenizer.tokenize(test_txt)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text) 
    return indexed_tokens

In [11]:
class AttentionBlock(nn.Module):
    def __init__(self, key_dim, val_dim, query_dim, hidden_dim, num_heads,FC_hidden=80):
        super(AttentionBlock, self).__init__()
        self.key_gen = nn.Sequential(
            nn.Linear(key_dim, 80),
            nn.ReLU(),
            nn.Linear(80, 80),
            nn.ReLU(),
            nn.Linear(80, hidden_dim),
            nn.ReLU(),
        )

        self.val_gen = nn.Sequential(
            nn.Linear(val_dim, 80),
            nn.ReLU(),
            nn.Linear(80, 80),
            nn.ReLU(),
            nn.Linear(80, hidden_dim),
            nn.ReLU(),
        )
        
        self.query_gen = nn.Sequential(
            nn.Linear(query_dim, 80),
            nn.ReLU(),
            nn.Linear(80, 80),
            nn.ReLU(),
            nn.Linear(80, hidden_dim),
            nn.Softmax(dim=-1),
        )
        self.multihead_attn = nn.MultiheadAttention(hidden_dim, num_heads, batch_first=True)
    
    def forward(self,keys,values,queries):
        key = self.key_gen(keys) #generate key with FC key is directly the embedding
        value = self.val_gen(values) #generate value with FC
        query = self.query_gen(queries) #generate query with FC
        output, _ =  self.multihead_attn(key=key, value=value, query=query)
        return output

In [40]:
model = attentive_model_pepemarti(hidden_dim=30,lstm_nl=1,target_dim=1,nheads=1,bert=bert)
model = model.to(device)

In [37]:
def train(dataloader, model, batch_size, num_epochs,lr=0.001,y_idx=0):
    model.train()

    criterion =  nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    losses = []
    for epoch in range(num_epochs):
        tqdm_aux = tqdm(enumerate(dataloader),total=len(dataloader))
        for batch, data in tqdm_aux:
            ocu_emb, espai_enc, general_data = data["activ:"] ,data["espai"] ,data["general_data"].float().to(device)
            y = data["y"][:,y_idx].float().to(device) #we'll do one counter for now
            optimizer.zero_grad()

            #current batch size size
            b_sz = len(ocu_emb[0])

            #note the dataloader with a batch of 100 when reachs the end expects a batch of 60
            h, c = model.init_hidden(b_sz) # Start with a new state in each batch            
            h = h.to(device)
            c = c.to(device)
            y_pred, h,c= model(ocu_emb, espai_enc, general_data, h, c)
            y_pred = y_pred[:,0,0]
            loss = criterion(y_pred,y)  #cross entropy loss needs (N,C,seq_lenght)
            loss.backward()
            optimizer.step()

            #tqdm_aux.set_description(f"Loss {loss.item()}")
            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })
            losses.append(loss.item())
    return losses

In [41]:
num_epochs = 20
losses = train(dataloader, model, batch_size=batch_size, num_epochs=num_epochs,lr=0.0001,y_idx=2)

  0%|          | 0/876 [00:00<?, ?it/s]ic| i: 4
ic| l: 4
ic| i: 9
ic| l: 4
ic| i: 14
ic| l: 4
ic| i: 19
ic| l: 4
ic| i: 24
ic| l: 4
ic| i: 29
ic| l: 4
ic| i: 34
ic| l: 4
ic| i: 39
ic| l: 4
ic| i: 44
ic| l: 4
ic| i: 49
ic| l: 4
ic| i: 4
ic| l: 4
ic| i: 9
ic| l: 4
ic| i: 14
ic| l: 4
ic| i: 19
ic| l: 4
ic| i: 24
ic| l: 4
ic| i: 29
ic| l: 4
ic| i: 34
ic| l: 4
ic| i: 39
ic| l: 4
ic| i: 44
ic| l: 4
ic| i: 49
ic| l: 4
ic| i: 4
ic| l: 4
ic| i: 9
ic| l: 4
ic| i: 14
ic| l: 4
ic| i: 19
ic| l: 4
ic| i: 24
ic| l: 4
ic| i: 29
ic| l: 4
ic| i: 34
ic| l: 4
ic| i: 39
ic| l: 4
ic| i: 44
ic| l: 4
ic| i: 49
ic| l: 4
ic| i: 4
ic| l: 4
ic| i: 9
ic| l: 4
ic| i: 14
ic| l: 4
ic| i: 19
ic| l: 4
ic| i: 24
ic| l: 4
ic| i: 29
ic| l: 4
ic| i: 34
ic| l: 4
ic| i: 39
ic| l: 4
ic| i: 44
ic| l: 4
ic| i: 49
ic| l: 4
ic| i: 4
ic| l: 4
ic| i: 9
ic| l: 4
ic| i: 14
ic| l: 4
ic| i: 19
ic| l: 4
ic| i: 24
ic| l: 4
ic| i: 29
ic| l: 4
ic| i: 34
ic| l: 4
ic| i: 39
  0%|          | 0/876 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
scaler = dataset.column_scalers[dataset.target_labels[2]]
#get the simple loss back from the squared error
scaled_losses = np.ravel(np.power(dataset.denormalize_values(np.sqrt(losses),scaler),2))
plt.plot(scaled_losses[5:])
plt.title('model L2 loss without normalisation')
plt.show()

NameError: name 'losses' is not defined

In [ ]:
def predict_consumption(model,dataloader,y_idx):
    real = []
    pred = []
    h, c = model.init_hidden(dataloader.batch_size) # Start with a new state in each batch            
    h = h.to(device)
    c = c.to(device)
    for batch, data in tqdm(enumerate(dataloader),total=len(dataloader)):
        with torch.no_grad():
            ocu_emb, espai_enc, general_data = data["activ:"] ,data["espai"] ,data["general_data"].float().to(device)
            y = data["y"][:,y_idx].float().to(device) #we'll do one counter for now

            #current batch size size
            b_sz = len(ocu_emb[0])

            #note the dataloader with a batch of 100 when reachs the end expects a batch of 60
            h, c = model.init_hidden(b_sz) # Start with a new state in each batch            
            h = h.to(device)
            c = c.to(device)
            y_pred, h,c= model(ocu_emb, espai_enc, general_data, h, c)
            y_pred = y_pred[:,0,0]
            pred += list(y_pred.to("cpu"))
            real += list(y.to("cpu"))
    return real ,pred

In [ ]:
real, pred = predict_consumption(model,dataloader,y_idx=2)
real = [v.item() for v in real]
pred = [v.item() for v in pred]
scaler = dataset.column_scalers[dataset.target_labels[0]]
real_sc = dataset.denormalize_values(real,scaler)
pred_sc = dataset.denormalize_values(pred,scaler)
plt.figure(figsize=(30, 6))  # Increase the figure size to make it larger in the x-axis

plt.plot(real_sc, label='Real')  # Add a label for the real data
plt.plot(pred_sc, label='Predicted')  # Add a label for the predicted data

plt.legend()  # Add a legend to the plot

plt.show()  # Display the plot

  0%|          | 0/292 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  5%|▌         | 15/292 [00:52<16:17,  3.53s/it]


KeyboardInterrupt: 